In [ ]:
#!/usr/bin/python
#import nglview
import sys
import itertools
import numpy as np
import pickle
import math
import matplotlib.pylab as plt
import scipy as sp
import iotbx.pdb
import cctbx
import pandas as pd
import seaborn as sns
import pymc3 as pm
import theano.tensor as T
import matplotlib

#from theano.compile.ops import as_op
import mmtbx.model
from scipy.optimize import fmin, fmin_cg, leastsq, fmin_l_bfgs_b
from scipy.interpolate import interp1d
import scipy.stats 
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
sns.set_palette("colorblind")
from cctbx import uctbx
from cctbx import crystal

In [ ]:
def adp2mtx(U11,U22,U33,U12,U13,U23):
    UIJ=np.zeros((3,3))
    UIJ[0,0]=U11
    UIJ[1,1]=U22
    UIJ[2,2]=U33
    UIJ[0,1]=U12
    UIJ[1,0]=U12
    UIJ[0,2]=U13
    UIJ[2,0]=U13
    UIJ[2,1]=U23
    UIJ[1,2]=U23
    return np.matrix(UIJ)

In [ ]:
def calc_anis(U11,U22,U33,U12,U13,U23):
    UIJ=np.zeros((3,3))
    UIJ[0,0]=U11
    UIJ[1,1]=U22
    UIJ[2,2]=U33
    UIJ[0,1]=U12
    UIJ[1,0]=U12
    UIJ[0,2]=U13
    UIJ[2,0]=U13
    UIJ[2,1]=U23
    UIJ[1,2]=U23
    return sp.linalg.eigh(UIJ)[0]

def test_det(U11,U22,U33,U12,U13,U23):
    UIJ=np.zeros((3,3))
    UIJ[0,0]=U11
    UIJ[1,1]=U22
    UIJ[2,2]=U33
    UIJ[0,1]=U12
    UIJ[1,0]=U12
    UIJ[0,2]=U13
    UIJ[2,0]=U13
    UIJ[2,1]=U23
    UIJ[1,2]=U23
    if np.all(np.linalg.eigvals(UIJ) > 0):
        ret='ok'
    else:
        ret='badegg!'
        print(ret)
    return ret


def read_data(oddst, xtal):
    global df
    odd = iotbx.pdb.input(file_name=oddst)

    for atm_odd in odd.atoms_with_labels():
        if not atm_odd.resname=="HOH" and not atm_odd.element==" H":
            if test_det(*atm_odd.uij)=='ok':
                oddeigval=calc_anis(*atm_odd.uij)
                oddb_est=8.0*np.power(np.pi,2.0)*np.sqrt(np.sum(oddeigval)/np.sum(np.reciprocal(oddeigval)))
                df=df.append({'b':atm_odd.b,
                          'b_est': oddb_est,
                          'U11':atm_odd.uij[0],
                          'U22':atm_odd.uij[1],
                          'U33':atm_odd.uij[2],
                          'U12':atm_odd.uij[3],
                          'U13':atm_odd.uij[4],
                          'U23':atm_odd.uij[5],
                          'anis':np.min(oddeigval)/np.max(oddeigval),
                          'late':((oddeigval[1]-np.min(oddeigval))/(np.max(oddeigval)-np.min(oddeigval))),
                          'x': atm_odd.xyz[0],
                          'y': atm_odd.xyz[1],
                          'z': atm_odd.xyz[2],
                          'chain': atm_odd.chain_id,
                          'resname': atm_odd.resname,
                          'xtal': xtal,
                          'resnum': int(atm_odd.resseq),
                          'combo': atm_odd.icode+atm_odd.resseq+atm_odd.altloc,
                          'altloc': atm_odd.altloc,
                          'Residue_label': atm_odd.resseq+atm_odd.icode+atm_odd.altloc+' '+atm_odd.resname,
                          'Residue_label2': atm_odd.chain_id+atm_odd.icode+atm_odd.resseq+' '+atm_odd.resname,
                          'Atom_label': atm_odd.resseq+atm_odd.icode+atm_odd.altloc+' '+atm_odd.resname+atm_odd.name,
                          'atmname': atm_odd.name },ignore_index=True)

                          
    return df, odd.crystal_symmetry()


df = pd.DataFrame()

root="./"

df,space_group=read_data("pdbs/2xtt.pdb",'x')

df=df[(df.altloc==' ')|(df.altloc=='A')]


In [ ]:
line1=np.array(df[['U11','U12','U13']])
line2=np.array(df[['U12','U22','U23']])
line3=np.array(df[['U13','U23','U33']])
lendf=len(df)
bigmtx=np.zeros((lendf,3,3))
for i in range(0,lendf):
    bigmtx[i]=np.array([line1[i],line2[i],line3[i]])
choice=np.random.choice(range(0,bigmtx.shape[0]), size=150, replace=False)

In [ ]:
eye = np.eye(3)
with pm.Model() as model:
    
    nu_offset = pm.HalfNormal('ν₀', 3)
    nu = pm.Deterministic('ν', 5 + nu_offset)
    V_p = pm.WishartBartlett('V', nu=5, S=eye, is_cholesky=True)
    for hck in choice:
        w2 = pm.Wishart('w%d' % hck, nu=nu, V=V_p/nu, shape=(3,3),observed=bigmtx[hck])
    trace2 = pm.sample(1000, tune=2000, cores=4, chains=4, progressbar=True)

In [ ]:
with model:
    trace=pm.sample_posterior_predictive(trace=trace2,samples=150)
trw=list(trace.keys())[0]